*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Dense, Embedding
print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

Load Data

In [ ]:
train_df = pd.read_table('/content/train-data.tsv',names = ["label","message"])
test_df = pd.read_table('/content/valid-data.tsv',names = ["label","message"])

In [ ]:
train_df.tail()

In [ ]:
test_df.head()

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

convert label to numerical variable

In [ ]:
train_df['label']= pd.get_dummies(train_df['label'])
test_df['label'] = pd.get_dummies(test_df['label'])


In [ ]:
train_df.tail()

Put data in appropriate structure

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_df["message"].values, train_df["label"].values))
test_dataset = tf.data.Dataset.from_tensor_slices((test_df["message"].values, test_df["label"].values))

print(train_dataset.element_spec,test_dataset.element_spec)

In [ ]:
# prefetching / buffering of the data for best performance on GPU
train_dataset = train_dataset.shuffle(buffer_size =100).batch(batch_size = 30).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size = 30).prefetch(tf.data.AUTOTUNE)

Text Preprocessing : TextVectorization

In [ ]:
# constants.
max_features = 1000
embedding_dim = 50
sequence_length = 100

#define vetorization layer
vectorize_layer = TextVectorization(
max_tokens=max_features,
output_mode='int',
output_sequence_length=sequence_length)

text_ds = train_dataset.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)




Build Model

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.Input(shape=(1,), dtype=tf.string),
  vectorize_layer,
  tf.keras.layers.Embedding(max_features + 1, embedding_dim),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])
  

In [ ]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(input_dim= max_features, 
                           output_dim=embedding_dim, 
                           input_length=sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'])
model.summary()

Fit model and plot history

In [ ]:
history = model.fit(train_dataset,
                    epochs=10,
                    verbose=False,
                    validation_data=test_dataset,
                    validation_steps=20,
                    batch_size=10)


In [ ]:
loss, accuracy = model.evaluate(train_dataset, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_dataset, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


plot train test error

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  
  prediction = model.predict(pred_text)

  print(prediction)

  if np.argmax(prediction[0].item()) > 0.5:
    label = "ham"
  else:
    label = "spam"

  prediction = [prediction[0].item(),label]
  
  return prediction

pred_text = "sale today! to stop texts call 98912460324",
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
